In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
mistral_api_key = os.environ["MISTRAL_API_KEY"]

In [2]:
from langchain_mistralai import ChatMistralAI

chatModel = ChatMistralAI(
    mistral_api_key=mistral_api_key,
    model="mistral-large-latest",
    temperature=0.7,
    max_retries=2,
    # other params...
)

In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader(
    web_paths=("https://www.gsmarena.com/apple_iphone_16-review-2749.php#",
               "https://www.wired.com/review/apple-iphone-16-and-iphone-16-plus/",
               "https://www.gadgets360.com/mobiles/reviews/iphone-16-pro-max-review-6702737",
                "https://www.stuffindia.in/reviews/apple-iphone-16-review/548",
              "https://www.gadgets360.com/internet/features/flipkart-big-diwali-sale-2024-iphone-13-14-15-16-plus-pro-max-deals-6855399",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("review-body clearfix", "article-blurb article-blurb-findings", "body__inner-container", "sdesc", 
                    "fullstoryCtrl_fulldetails", "post-item-layout", "content_text row description")
        )
    ),
)

In [5]:
docs = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)

In [7]:
splits = text_splitter.split_documents(docs)

In [8]:
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

/Users/pramodmodi/Library/Caches/pypoetry/virtualenvs/langchain-mistral-xuCIEomg-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

#prompt = hub.pull("rlm/rag-prompt")

In [10]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

In [13]:
query= "What are new feature in iphone16 in comparision to iphone15?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Based on the provided context, the new features in the iPhone 16 compared to the iPhone 15 are:

1. **Camera Improvements:**
   - 48MP primary camera with sensor-shift stabilization and Fusion capability for improved 2x in-sensor zoom.
   - Upgraded ultrawide camera with a brighter aperture lens and autofocus for macro photos.
   - New Tone setting for fine-tuning the camera's tonal processing.

2. **Wireless Charging:**
   - Next-generation MagSafe and Qi2 support for up to 25W wireless charging with a new MagSafe charger and a 30W+ power adapter.

3. **New Buttons:**
   - Action button and Camera Control button.

4. **Performance:**
   - Apple A18 chip with a 5-core GPU capable of Ray Tracing and improved gaming and video rendering performance (40% increase over the iPhone 15).

5. **Software:**
   - Runs on iOS 18 and supports Apple Intelligence (though it has yet to be released).

6. **Design:**
   - More buttons than any other current smartphone, contrary to rumors of a portless o

In [123]:
query= "Best deals for iphone 16 pro max, iphone 16 pro, iphone 15, iphone 15 pro max. Give result in tabular format"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context, here are the best deals for the mentioned iPhones in a tabular format:

| iPhone Model | Original Price (Rs.) | Sale Price (Rs.) | Additional Benefits |
|---|---|---|---|
| iPhone 16 Pro Max (256GB) | 1,44,900 | 1,30,410 | 10% instant discount for SBI card and Mastercard holders, 5% cashback for Flipkart Axis Bank Credit Card users, exchange offers |
| iPhone 16 Pro | Not explicitly mentioned | Not explicitly mentioned | 10% instant discount for SBI card and Mastercard holders, 5% cashback for Flipkart Axis Bank Credit Card users, exchange offers |
| iPhone 15 | Not explicitly mentioned | Not explicitly mentioned | 10% instant discount for SBI card and Mastercard holders, 5% cashback for Flipkart Axis Bank Credit Card users, exchange offers |
| iPhone 15 Pro Max | Not explicitly mentioned | Not explicitly mentioned | 10% instant discount for SBI card and Mastercard holders, 5% cashback for Flipkart Axis Bank Credit Card users, exchange offers |

Note:
- T

In [14]:
query= "Whay should I purchase iphone16? and what is the price"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context, you should consider purchasing the iPhone 16 for the following reasons:

1. **Great performance**: It has a new, more powerful processor and much more advanced graphics with the Apple A18 chip. It's excellent for gaming and general performance.
2. **Improved cameras**: The iPhone 16 comes with camera improvements, although it doesn't have the third telephoto camera like the Pro models.
3. **Stunning display**: It has a 6.1-inch Super Retina XDR OLED display with high brightness, HDR10, and Dolby Vision.
4. **Sturdiness**: The iPhone 16 is a pioneer in sturdiness thanks to an improved Ceramic Shield glass and has an IP68 rating for up to 6m water submersion.
5. **Battery life**: Although the context mentions average battery life, it also states that the iPhone 16 Plus has "bonkers" battery life, suggesting that the iPhone 16 also has decent battery performance.

The price of the iPhone 16, based on the provided context, is:

- 128GB 8GB RAM: ₹79,900 or $82

In [15]:
query= "Give me more details about battery, cammera, charging, wireless file transfer and performance of Iphone 16 ?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context, here are more details about the battery, camera, charging, wireless file transfer, and performance of the iPhone 16:

**Battery:**
- The iPhone 16 offers around six hours of screen-on time with about 20% remaining after a day of mixed usage (including social media, photography, navigation, and music streaming).
- The battery is easier to replace on the iPhone 16 and 16 Plus, with iFixit giving it a 7/10 in overall repairability.
- The device is expected to last a full day and then some on a single charge.

**Camera:**
- The iPhone 16 features a dual-camera system on the back:
  - A 48MP primary camera with sensor-shift stabilization and Fusion capability, allowing for improved 2x in-sensor zoom (better than the iPhone 15).
  - An upgraded ultrawide camera with a brighter aperture lens and autofocus for macro photos.
- It also introduces a new Tone setting to fine-tune the camera's tonal processing to match the user's taste.
- The camera has been moved to 

In [16]:
query= "WHat are specification of Iphone 16 ?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context, here are the specifications of the iPhone 16:

- **Battery:**
  - Capacity: 3561mAh
  - Wired charging: PD2.0, 50% in 30 min (advertised)
  - Wireless charging: 25W (MagSafe), 15W (China only), 15W (Qi2)
  - Reverse wired charging: 4.5W

- **Connectivity:**
  - 5G, eSIM, Dual SIM, Wi-Fi 7, Bluetooth 5.3, NFC

- **Other Features:**
  - Face ID, accelerometer, gyro, proximity, compass, barometer
  - Stereo speakers
  - Ultra Wideband (UWB) support (gen2 chip)
  - Emergency SOS, Messages and Find My via satellite

- **Display:**
  - 60Hz screen refresh rate

- **Camera:**
  - Dual camera: 48MP primary (with sensor-shift stabilization and Fusion capability), upgraded ultrawide camera (brighter aperture lens and autofocus)
  - New Tone setting for camera tonal processing

- **Software:**
  - iOS 18 with support for Apple Intelligence

- **Charging:**
  - Next-generation MagSafe and Qi2 support for up to 25W wireless charging

- **In the Box:**
  - Braided USB-

In [17]:
query= "WHat are specification of Iphone 16 pro max?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context, here are the specifications of the iPhone 16 Pro Max:

- **Display**: 6.90-inch with a resolution of 1320x2868 pixels at 460ppi pixel density, 2000 nits peak brightness
- **Processor**: Apple A18 Pro (six-core CPU with six-core GPU and 16-core Neural Engine)
- **Front Camera**: 12-megapixel
- **Rear Camera**: 48-megapixel + 12-megapixel + 48-megapixel
- **RAM**: 8GB
- **Storage**: 256GB, 512GB, 1TB
- **OS**: iOS 18
- **Battery Life**: Not explicitly stated, but mentioned as "fantastic"
- **Other Features**:
  - Massive screen size
  - Brilliant display
  - Performance beast
  - Camera Control is a boon
  - Expensive
  - No Apple Intelligence at launch
  - Slow-wired charging support

In [109]:
query= "What about camera control button and action button of Iphone 16?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

The iPhone 16 series includes two notable buttons: the Camera Control and the Action Button.

1. **Camera Control**: This is a dedicated camera control button available on all iPhone 16 models. It supports various touch inputs such as soft and hard presses, as well as swipe gestures to control different camera functions. It also supports acceleration, allowing for quick scrolling based on the speed of the swipe gesture.

2. **Action Button**: This button comes from the Pro models of the previous year and replaces the Alert Slider on the top left. It can be customized as a macro key to launch any desired function or app.

In total, the iPhone 16 series has six buttons, with the iPhone 16 Pro Max specifically having the power and volume rockers alongside the Camera Control and Action buttons, making four physical buttons overall.

In [110]:
query= "Iphone 16 vs Iphone 16 pro max?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context, here are the key differences between the iPhone 16 and iPhone 16 Pro Max:

1. **Display**:
   - iPhone 16 Pro Max: Has a 120Hz refresh rate display.
   - iPhone 16: Does not have a 120Hz refresh rate display.

2. **Camera**:
   - iPhone 16 Pro Max: Has a third telephoto camera.
   - iPhone 16: Does not have a third telephoto camera.

3. **Size**:
   - iPhone 16 Pro Max: Is not a small phone, with the biggest display on an iPhone yet.
   - iPhone 16: No information is provided about its size, but it's assumed to be smaller than the Pro Max.

4. **Price**:
   - iPhone 16 Pro Max: More expensive, likely over Rs. 1 lakh.
   - iPhone 16: Less expensive, under Rs. 1 lakh.

5. **Battery and Charging**:
   - iPhone 16 Pro Max: Excellent battery performance but slower wired charging (full charge takes about 100-120 minutes with a 20W adapter).
   - iPhone 16: No specific information provided about battery performance or charging speed.

The context suggests that i

In [124]:
query= "Who is the prime minister of india today?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

The context provided does not contain information about the Prime Minister of India. Therefore, I cannot answer the question based on the given context.